In [ ]:
import pandas as pd
import numpy as np
import warnings
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import dash
from dash import dcc, html, dash_table, callback
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc


import socket
# Configuration Variables (Modify as needed)
file_path = "reportjun25.csv"  # Path to the CSV file
AOV = 1500  # Average Order Value
CHANNEL_COSTS = {
    'Email': 0.02,
    'Push': 0.01, 
    'SMS': 0.11,
    'WhatsApp': 0.11
}



class CampaignAnalyzer:
    def __init__(self, file_path, aov=AOV, channel_costs=CHANNEL_COSTS):
        self.file_path = file_path
        self.aov = aov
        self.channel_costs = channel_costs
        self.raw_data = None
        self.summary_data = None
        self.channel_data = None
        
    def load_and_process_data(self):
        """Load and process campaign data"""
        try:
            # Load the data
            df = pd.read_csv(self.file_path)
            
            # Basic data cleaning
            df = df.fillna(0)
            
            # Convert numeric columns
            numeric_columns = ['Sent', 'Delivered', 'Unique Impressions', 'Unique Clicks', 
                              'Unique Conversions', 'Unique Click-Through Conversions']
            
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0)
            
            self.raw_data = df
            return df
        except Exception as e:
            print(f"Error loading data: {e}")
            return None
    
    def calculate_summary_metrics(self):
        """Calculate main summary table metrics"""
        if self.raw_data is None:
            return None
            
        df = self.raw_data
        
        # Group by Journey Name for analysis
        grouped = df.groupby('Journey Name').agg({
            'Campaign ID': 'count',
            'Sent': 'sum',
            'Delivered': 'sum', 
            'Unique Impressions': 'sum',
            'Unique Clicks': 'sum',
            'Unique Conversions': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Rename columns for clarity
        grouped.rename(columns={'Campaign ID': 'Count of Campaign ID'}, inplace=True)
        
        # Calculate performance metrics
        grouped['v_dr'] = np.where(grouped['Sent'] > 0, 
                                  (grouped['Delivered'] / grouped['Sent'] * 100).round(0), 0)
        
        grouped['v_ctr'] = np.where(grouped['Delivered'] > 0,
                                   (grouped['Unique Clicks'] / grouped['Delivered'] * 100).round(1), 0)
        
        grouped['v_cr'] = np.where(grouped['Unique Clicks'] > 0,
                                  (grouped['Unique Click-Through Conversions'] / grouped['Unique Clicks'] * 100).round(1), 0)
        
        grouped['v_ord_per_sent'] = np.where(grouped['Sent'] > 0,
                                            (grouped['Unique Click-Through Conversions'] / grouped['Sent'] * 100).round(2), 0)
        
        # Calculate financial metrics
        financial_metrics = self.calculate_financial_metrics()
        
        # Merge financial data
        summary_df = grouped.merge(financial_metrics[['Journey Name', 'Cost', 'GTV', 'ROI']], 
                                 on='Journey Name', how='left')
        
        # Format the data for display
        summary_df['Row Labels'] = summary_df['Journey Name']
        summary_df['Sum of Sent'] = summary_df['Sent'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Delivered'] = summary_df['Delivered'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Impressions'] = summary_df['Unique Impressions'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Clicks'] = summary_df['Unique Clicks'].apply(lambda x: f"{int(x):,}")
        summary_df['Sum of Unique Click-Through Conversions'] = summary_df['Unique Click-Through Conversions'].apply(lambda x: f"{int(x):,}")
        summary_df['v_dr_formatted'] = summary_df['v_dr'].astype(int).astype(str) + '%'
        summary_df['v_ctr_formatted'] = summary_df['v_ctr'].astype(str) + '%'
        summary_df['v_cr_formatted'] = summary_df['v_cr'].astype(str) + '%'
        summary_df['v_ord_per_sent_formatted'] = summary_df['v_ord_per_sent'].astype(str) + '%'
        summary_df['Cost_formatted'] = summary_df['Cost'].apply(lambda x: f"₹ {x:,.2f}")
        summary_df['GTV_formatted'] = summary_df['GTV'].apply(lambda x: f"{int(x):,}")
        summary_df['ROI_formatted'] = summary_df['ROI'].apply(lambda x: f"₹ {x:.2f}")
        
        self.summary_data = summary_df
        return summary_df
    
    def calculate_financial_metrics(self):
        """Calculate cost, GTV, and ROI metrics"""
        if self.raw_data is None:
            return pd.DataFrame()
            
        df = self.raw_data
        
        financial_df = df.groupby('Journey Name').agg({
            'Sent': 'sum',
            'Channel': lambda x: list(x),
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Calculate costs based on channel
        financial_df['Cost'] = 0
        for idx, row in financial_df.iterrows():
            channel_sent = df[df['Journey Name'] == row['Journey Name']].groupby('Channel')['Sent'].sum()
            total_cost = 0
            for channel, sent_count in channel_sent.items():
                if channel in self.channel_costs:
                    total_cost += sent_count * self.channel_costs[channel]
            financial_df.at[idx, 'Cost'] = total_cost
        
        # Calculate GTV and ROI
        financial_df['GTV'] = financial_df['Unique Click-Through Conversions'] * self.aov
        financial_df['ROI'] = np.where(financial_df['Cost'] > 0,
                                      (financial_df['GTV'] / financial_df['Cost']).round(2), 0)
        
        return financial_df
    
    def calculate_channel_breakdown(self):
        """Calculate channel-wise breakdown"""
        if self.raw_data is None:
            return pd.DataFrame()
            
        df = self.raw_data
        
        # Create channel analysis
        channel_analysis = df.groupby(['Journey Name', 'Channel']).agg({
            'Sent': 'sum',
            'Delivered': 'sum',
            'Unique Click-Through Conversions': 'sum'
        }).reset_index()
        
        # Create a comprehensive breakdown table similar to your reference
        journeys = df['Journey Name'].unique()
        channels = ['Email', 'Push', 'SMS', 'WhatsApp']
        
        # Initialize the breakdown dataframe
        breakdown_data = []
        
        for journey in journeys:
            journey_data = df[df['Journey Name'] == journey]
            row_data = {'Row Labels': journey}
            
            # Calculate totals
            total_sent = journey_data['Sent'].sum()
            total_delivered = journey_data['Delivered'].sum()
            total_conversions = journey_data['Unique Click-Through Conversions'].sum()
            
            # Add channel-wise data
            for channel in channels:
                channel_data = journey_data[journey_data['Channel'] == channel]
                sent = channel_data['Sent'].sum()
                delivered = channel_data['Delivered'].sum()
                conversions = channel_data['Unique Click-Through Conversions'].sum()
                
                row_data[f'{channel}_Sent'] = f"{int(sent):,}" if sent > 0 else ""
                row_data[f'{channel}_Delivered'] = f"{int(delivered):,}" if delivered > 0 else ""
                row_data[f'{channel}_Conversions'] = f"{int(conversions):,}" if conversions > 0 else ""
            
            # Add totals
            row_data['Total_Sent'] = f"{int(total_sent):,}"
            row_data['Total_Delivered'] = f"{int(total_delivered):,}"
            row_data['Total_Conversions'] = f"{int(total_conversions):,}"
            
            breakdown_data.append(row_data)
        
        channel_breakdown = pd.DataFrame(breakdown_data)
        self.channel_data = channel_breakdown
        return channel_breakdown

# Initialize the analyzer
try:
    analyzer = CampaignAnalyzer("report-1750749443470_jBRS6FP_Snapmint _in~~c2ab3517.csv")
    
    # Load and process data
    data_loaded = analyzer.load_and_process_data()
    
    if data_loaded is not None:
        summary_df = analyzer.calculate_summary_metrics()
        channel_df = analyzer.calculate_channel_breakdown()
        
        print(f"✅ Data loaded successfully: {len(data_loaded)} records")
        print(f"📊 Summary table: {len(summary_df)} journeys")
        print(f"📋 Channel breakdown: {len(channel_df)} journeys")
    else:
        print("❌ Failed to load data")
        summary_df = pd.DataFrame()
        channel_df = pd.DataFrame()
        
except Exception as e:
    print(f"❌ Error initializing analyzer: {e}")
    summary_df = pd.DataFrame()
    channel_df = pd.DataFrame()

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout with error handling
def create_layout():
    if summary_df.empty:
        return dbc.Container([
            html.H1("Campaign Performance Analytics Dashboard", className="text-center mb-4 text-danger"),
            html.Hr(),
            dbc.Alert("Failed to load data. Please check your file path and data format.", color="danger")
        ], fluid=True)
    
    return dbc.Container([
        dbc.Row([
            dbc.Col([
                html.H1("Campaign Performance Analytics Dashboard", 
                       className="text-center mb-4 text-primary"),
                html.Hr()
            ])
        ]),
        
        # Key Metrics Row
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Sent'].sum():,}", className="text-primary"),
                        html.P("Total Messages Sent", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Delivered'].sum():,}", className="text-success"),
                        html.P("Total Delivered", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{summary_df['Unique Click-Through Conversions'].sum():,}", className="text-info"),
                        html.P("Total Conversions", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{summary_df['Cost'].sum():,.2f}", className="text-warning"),
                        html.P("Total Cost", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"₹{summary_df['GTV'].sum():,.0f}", className="text-success"),
                        html.P("Total GTV", className="mb-0")
                    ])
                ])
            ], width=2),
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H4(f"{(summary_df['GTV'].sum()/summary_df['Cost'].sum() if summary_df['Cost'].sum() > 0 else 0):.2f}x", className="text-danger"),
                        html.P("Overall ROI", className="mb-0")
                    ])
                ])
            ], width=2),
        ], className="mb-4"),
        
        # Tabs for different views
        dbc.Tabs([
            dbc.Tab(label="Summary Table", tab_id="summary"),
            dbc.Tab(label="Channel Breakdown", tab_id="channels"),
            dbc.Tab(label="Performance Charts", tab_id="charts"),
        ], id="tabs", active_tab="summary"),
        
        html.Div(id="tab-content", className="mt-4")
    ], fluid=True)

app.layout = create_layout()

@app.callback(
    Output("tab-content", "children"),
    Input("tabs", "active_tab")
)
def render_tab_content(active_tab):
    if summary_df.empty:
        return dbc.Alert("No data available to display.", color="warning")
    
    try:
        if active_tab == "summary":
            # Create summary table
            summary_display_df = summary_df[[
                'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
                'Sum of Unique Impressions', 'Sum of Unique Clicks', 
                'Sum of Unique Click-Through Conversions', 'v_dr_formatted', 
                'v_ctr_formatted', 'v_cr_formatted', 'v_ord_per_sent_formatted',
                'Cost_formatted', 'GTV_formatted', 'ROI_formatted'
            ]].copy()
            
            summary_display_df.columns = [
                'Row Labels', 'Count of Campaign ID', 'Sum of Sent', 'Sum of Delivered',
                'Sum of Unique Impressions', 'Sum of Unique Clicks', 
                'Sum of Unique Click-Through Conversions', 'v_dr', 'v_ctr', 'v_cr',
                'v_ord_per_sent', 'Cost', 'GTV', 'ROI'
            ]
            
            return dash_table.DataTable(
                data=summary_display_df.to_dict('records'),
                columns=[{"name": i, "id": i} for i in summary_display_df.columns],
                style_table={'overflowX': 'auto'},
                style_cell={
                    'textAlign': 'left',
                    'padding': '10px',
                    'fontFamily': 'Arial',
                    'fontSize': '12px'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold',
                    'fontSize': '14px'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }
                ],
                sort_action="native",
                filter_action="native",
                page_size=20,
                export_format="xlsx",
                export_headers="display"
            )
        
        elif active_tab == "channels":
            if channel_df.empty:
                return dbc.Alert("No channel breakdown data available.", color="warning")
                
            return dash_table.DataTable(
                data=channel_df.to_dict('records'),
                columns=[{"name": i, "id": i} for i in channel_df.columns],
                style_table={'overflowX': 'auto'},
                style_cell={
                    'textAlign': 'left',
                    'padding': '8px',
                    'fontFamily': 'Arial',
                    'fontSize': '11px',
                    'minWidth': '80px'
                },
                style_header={
                    'backgroundColor': 'rgb(230, 230, 230)',
                    'fontWeight': 'bold',
                    'fontSize': '12px'
                },
                style_data_conditional=[
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': 'rgb(248, 248, 248)'
                    }
                ],
                sort_action="native",
                filter_action="native",
                page_size=15,
                export_format="xlsx",
                export_headers="display"
            )
        
        elif active_tab == "charts":
            # Create performance charts with error handling
            try:
                # Filter data for meaningful charts
                chart_data = summary_df[summary_df['ROI'] > 0].nlargest(10, 'ROI')
                
                if chart_data.empty:
                    return dbc.Alert("No data available for charts.", color="warning")
                
                # Chart 1: Top ROI Performance
                fig1 = px.bar(chart_data, 
                             x='Journey Name', y='ROI', 
                             title="Top 10 Journeys by ROI",
                             labels={'ROI': 'Return on Investment'})
                fig1.update_layout(xaxis_tickangle=45, height=400)
                
                # Chart 2: Cost vs GTV Analysis
                scatter_data = summary_df[(summary_df['Cost'] > 0) & (summary_df['GTV'] > 0)]
                fig2 = px.scatter(scatter_data, 
                                 x='Cost', y='GTV', 
                                 size='Unique Click-Through Conversions',
                                 hover_name='Journey Name',
                                 title="Cost vs GTV Analysis",
                                 labels={'Cost': 'Total Cost (₹)', 'GTV': 'Gross Transaction Value (₹)'})
                fig2.update_layout(height=400)
                
                # Chart 3: Volume Distribution
                volume_data = summary_df.nlargest(8, 'Sent')
                fig3 = px.pie(volume_data, 
                             values='Sent', names='Journey Name',
                             title="Top 8 Journeys by Messages Sent")
                fig3.update_layout(height=400)
                
                # Chart 4: Channel Performance
                if not channel_df.empty:
                    # Calculate channel totals
                    channel_totals = {}
                    for channel in ['Email', 'Push', 'SMS', 'WhatsApp']:
                        total = 0
                        for _, row in channel_df.iterrows():
                            val = row.get(f'{channel}_Sent', '0')
                            if val and val != '':
                                total += int(val.replace(',', ''))
                        channel_totals[channel] = total
                    
                    fig4 = px.bar(x=list(channel_totals.keys()), 
                                 y=list(channel_totals.values()),
                                 title="Messages Sent by Channel",
                                 labels={'x': 'Channel', 'y': 'Total Messages Sent'})
                    fig4.update_layout(height=400)
                else:
                    fig4 = px.bar(title="Channel data not available")
                
                return dbc.Row([
                    dbc.Col([dcc.Graph(figure=fig1)], width=6),
                    dbc.Col([dcc.Graph(figure=fig2)], width=6),
                    dbc.Col([dcc.Graph(figure=fig3)], width=6),
                    dbc.Col([dcc.Graph(figure=fig4)], width=6)
                ])
                
            except Exception as e:
                return dbc.Alert(f"Error creating charts: {str(e)}", color="danger")
    
    except Exception as e:
        return dbc.Alert(f"Error rendering content: {str(e)}", color="danger")
    
def get_local_ip():
    """Get the local IP address of the machine"""
    try:
        # Connect to a remote server to determine local IP
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        local_ip = s.getsockname()[0]
        s.close()
        return local_ip
    except Exception:
        return "localhost"
warnings.filterwarnings('ignore')


if __name__ == "__main__":
    local_ip = get_local_ip()
    
    print("🚀 Starting Campaign Analytics Dashboard...")
    print("📊 Dashboard URLs:")
    print(f"   - Local Access: http://127.0.0.1:8052")
    print(f"   - Network Access: http://{local_ip}:8052")
    print(f"   - WiFi Share URL: http://{local_ip}:8052")
    print("\n📋 Features available:")
    print("   - Interactive Summary Table (sortable, filterable, exportable)")
    print("   - Channel Breakdown Analysis")
    print("   - Performance Visualization Charts")
    print("   - Real-time metrics calculations")
    print(f"\n🌐 Share this URL with others on your WiFi:")
    print(f"   http://{local_ip}:8052")
    print("\n" + "="*60)
    
    try:
        # Use run_server instead of run, and specify the correct parameters
        app.run(
            debug=False,  # Set to False for network access
            host='0.0.0.0',  # Listen on all interfaces
            port=8052,  # Your chosen port
            dev_tools_hot_reload=False  # Disable hot reload for network access
        )
    except Exception as e:
        print(f"❌ Error starting server: {e}")
        print("📝 Trying alternative configuration...")
        try:
            # Fallback: Run on local IP only
            app.run(
                debug=False,
                host=local_ip,
                port=8052
            )
        except Exception as e2:
            print(f"❌ Fallback failed: {e2}")
            print("🔧 Try running on localhost only:")
            app.run(debug=True, host='127.0.0.1', port=8052)

✅ Data loaded successfully: 28750 records
📊 Summary table: 21 journeys
📋 Channel breakdown: 21 journeys
🚀 Starting Campaign Analytics Dashboard...
📊 Dashboard URLs:
   - Local Access: http://127.0.0.1:8052
   - Network Access: http://172.16.56.59:8052
   - WiFi Share URL: http://172.16.56.59:8052

📋 Features available:
   - Interactive Summary Table (sortable, filterable, exportable)
   - Channel Breakdown Analysis
   - Performance Visualization Charts
   - Real-time metrics calculations

🌐 Share this URL with others on your WiFi:
   http://172.16.56.59:8052

❌ Error starting server: HTTPConnectionPool(host='0.0.0.0', port=8052): Max retries exceeded with url: /_alive_df0f33a4-992f-48fd-b1a8-2ed130e9da73 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000016A4C2138C0>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))
📝 Trying alternative configuration...


: 